In [1]:
import sys
import os

import torch

# Assuming your project directory is one level up from the Jupyter notebook
sys.path.insert(0, os.path.abspath('../..'))

import lightning as L
import matplotlib
import numpy as np
import pandas as pd
import multiprocessing
from src.pipelines.trainers.trainerWrapper import TrainerWrapper
from src.util.conditional_early_stopping import ConditionalEarlyStopping
from src.util.power_splitter import PowerSplitter
from src.util.error import NRMSE
from src.util.plotly import plot_results
from src.util.flex_error import get_mafe, get_prob_mafe

from src.pipelines.cleaners.temp_cleaner import TempCleaner
from src.pipelines.models.lstm import LSTM
from src.pipelines.normalizers.min_max_normalizer import MinMaxNormalizer
from src.pipelines.sequencers.time_sequencer import TimeSequencer
from src.pipelines.splitters.std_splitter import StdSplitter
from src.pipelines.tuners.std_tuner_wrapper import StdTunerWrapper
from src.pipelines.optimizers.optimizer import OptimizerWrapper

from src.pipelines.deterministic_pipeline import DeterministicPipeline
from src.pipelines.monte_carlo_pipeline import MonteCarloPipeline
from src.pipelines.ensemble_pipeline import EnsemblePipeline
from src.pipelines.probabilistic_pipeline import ProbabilisticPipeline

import torch.optim as optim

matplotlib.use("Agg")

MODEL_PATH = 'model_saves/ex9_model'
NUM_WORKERS = multiprocessing.cpu_count()
TARGET_COLUMN = 2
TIMESTAMP = "Timestamp"
POWER     = "PowerConsumption"

# Hyper parameters
# Model
input_size = 4
time_horizon = 4
hidden_size = 32
num_epochs = 1000
seq_len = 96
num_layers = 2
 
# MC ONLY
inference_samples = 50

# Training
dropout = 0.50
gradient_clipping = 0
early_stopping_threshold = 0.15

num_ensembles = 5

# Flexibility
flex_confidence = 0.90
temp_boundary = 0.1
error = 0

# Controlled by tuner
batch_size = 128
learning_rate = 0.005

# Data Split
train_days = 16
val_days = 2
test_days = 2

# ON / OFF Power Limits
off_limit_w = 100
on_limit_w = 1500

consecutive_points = 3

nist_path = "../../src/data_preprocess/dataset/NIST_cleaned.csv"

clean_in_low = 10
clean_in_high = 30
clean_out_low = -50
clean_out_high = 50
clean_pow_low = 0
clean_delta_temp = 15

In [2]:
assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(TimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .build()

model = EnsemblePipeline.Builder() \
        .set_pipeline(model) \
        .set_num_ensembles(num_ensembles) \
        .build()

model.fit()
model.save(MODEL_PATH)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Restored all states from the checkpoint at /home/vind/P5/experimentation/experiments/iter1/.scale_batch_size_6e7ea6b0-1f17-409e-8d4d-ccfdcca4e008.ckpt

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | LSTM | 13.4 K | train
---------------------------------------
13.4 K    Trainable params
0         Non-trainable params
13.4 K    Total params
0.054     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Restored all states from the checkpoint at /home/vind/P5/experimentation/experiments/iter1/.scale_batch_size_38ec2d06-34e2-49d7-9e25-3ad48df3c3d7.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | LSTM | 13.4 K | train
---------------------------------------
13.4 K    Trainable params
0         Non-trainable params
13.4 K    Total params
0.054     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


...

Epoch 1:   0%|          | 0/53 [00:00<?, ?it/s, v_num=59, train_loss_step=0.348, val_loss=0.263, train_loss_epoch=0.355]
Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1:   0%|          | 0/53 [00:00<?, ?it/s, v_num=59, train_loss_step=0.348, val_loss=0.263, train_loss_epoch=0.355]


  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | LSTM | 13.4 K | train
---------------------------------------
13.4 K    Trainable params
0         Non-trainable params
13.4 K    Total params
0.054     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Epoch 46:  45%|████▌     | 24/53 [00:01<00:01, 19.05it/s, v_num=59, train_loss_step=0.147, val_loss=0.242, train_loss_epoch=0.179]        
saving to model_saves/ex9_model


In [4]:

df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(TimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .build()

model = EnsemblePipeline.Builder() \
        .set_pipeline(model) \
        .set_num_ensembles(num_ensembles) \
        .build()

model.load(MODEL_PATH)
model.test()

plot_results(model.get_predictions(), model.get_actuals(), model.get_timestamps())

model.eval()

ps = PowerSplitter(splitter.get_test(cleaner.clean(df)), TIMESTAMP, POWER)

on_df = ps.get_mt_power(on_limit_w, consecutive_points)
off_df = ps.get_lt_power(off_limit_w, consecutive_points)

on_data = np.array(on_df)
off_data = np.array(off_df)

print("Calculating mafe...")
if (isinstance(model, ProbabilisticPipeline)):
    print("PROB MAFE ON:", get_prob_mafe(on_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN, flex_confidence))
    print("PROB MAFE OFF:", get_prob_mafe(off_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN, flex_confidence))
else:
    print("MAFE ON:", get_mafe(on_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN))
    print("MAFE OFF:", get_mafe(off_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Train

Testing: |          | 0/? [00:00<?, ?it/s]t/s]
Testing DataLoader 0:   0%|          | 0/7 [00:00<?, ?it/s]
Testing: |          | 0/? [00:00<?, ?it/s]
Testing: |          | 0/? [00:00<?, ?it/s]
Testing: |          | 6/? [00:00<00:00, 19.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        0.12074175477027893
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_

Calculating mafe...


PROB MAFE ON: 0.09090909361839294


PROB MAFE OFF: 0.8678160905838013
